# Mortality in the US by Demographics: Cleaning Data

Authors: Ivana Lin and Ricky Ma

## Loading and Cleaning Data

In [ ]:
import pandas as pd

In [ ]:
df05 = pd.read_csv("2005_data.csv", parse_dates=['current_data_year'],
                   low_memory=False)
df06 = pd.read_csv("2006_data.csv", parse_dates=['current_data_year'],
                  low_memory=False)
df07 = pd.read_csv("2007_data.csv", parse_dates=['current_data_year'],
                  low_memory=False)
df08 = pd.read_csv("2008_data.csv", parse_dates=['current_data_year'],
                  low_memory=False)
df09 = pd.read_csv("2009_data.csv", parse_dates=['current_data_year'],
                  low_memory=False)
df10 = pd.read_csv("2010_data.csv", parse_dates=['current_data_year'],
                  low_memory=False)
df11 = pd.read_csv("2011_data.csv", parse_dates=['current_data_year'],
                  low_memory=False)
df12 = pd.read_csv("2012_data.csv", parse_dates=['current_data_year'],
                  low_memory=False)
df13 = pd.read_csv("2013_data.csv", parse_dates=['current_data_year'],
                  low_memory=False)
df14 = pd.read_csv("2014_data.csv", parse_dates=['current_data_year'],
                  low_memory=False)
df15 = pd.read_csv("2015_data.csv", parse_dates=['current_data_year'],
                  low_memory=False)

We drop columns that are unneccesary, have minimal data, or have no clear meaning according to the data dictionary. 

In [ ]:
drop_entity_cols = ["entity_condition_" + str(i) for i in range(1, 21)]
drop_record_cols = ["record_condition_" + str(i) for i in range(1, 21)]
drop_cols = drop_entity_cols + drop_record_cols + ['number_of_entity_axis_conditions',
'number_of_record_axis_conditions', 'place_of_injury_for_causes_w00_y34_except_y06_and_y07_']
drop_cols12 = drop_cols + ['icd_code_10']
drop_cols = drop_cols + ['icd_code_10th_revision']

df05 = df05.drop(columns=drop_cols)
df06 = df06.drop(columns=drop_cols)
df07 = df07.drop(columns=drop_cols)
df08 = df08.drop(columns=drop_cols)
df09 = df09.drop(columns=drop_cols)
df10 = df10.drop(columns=drop_cols)
df11 = df11.drop(columns=drop_cols)
df12 = df12.drop(columns=drop_cols12)
df13 = df13.drop(columns=drop_cols)
df14 = df14.drop(columns=drop_cols)
df15 = df15.drop(columns=drop_cols)
df = df05.append([df06, df07, df08, df09, df10, df11, df12, df13, df14, df15])

We then convert column values from numerical mappings to strings representing their categories according to the data dictionary. This makes the data more readable for our visualizations.

#### Resident Status

In [ ]:
def convert_status(s):
    '''
    Converts the resident_status field in the dataframe from the
    encoding to the actual value. e.g. 1 => RESIDENTS.
    '''
    if (s == None):
        return None
    status = ["RESIDENTS", "INTRASTATE NONRESIDENTS", 
              "INTERSTATE NONRESIDENTS", "FOREIGN RESIDENTS"]
    return status[s-1]

df['resident_status'] = df['resident_status'].apply(convert_status)

#### Education

The categories of the 2003 and 1989 recodes are significantly different and we are unable to reconcile the differences between the two. In cases where the 1989 recode has "2 years of high school", for example, we have no way of converting that to the 2003 recode since we don't know if they got a GED or not. Thus we choose to stick with only one of the recodes.

In [ ]:
print(df[df['education_1989_revision'].notnull()].shape)
print(df[df['education_2003_revision'].notnull()].shape)

(8194939, 33)
(19525734, 33)


Since the data categories for the 2003 recode are more descriptive and inclusive according to the data dictionary, we will be retaining only the education_2003_revision column and dropping all rows that have null values for that column. Although we lose a significant amount of data, we have still have so much data remaining across all the years that it would still be enough to work with.

In [ ]:
def convert_education_2003(s):
    '''
    Converts the education status under the 2003 classification from the 
    encoding to the actual value. e.g. 1 => 8th grade or less.
    '''
    if (s == None or int(s) == 9):
        return None
    edu = ["8th grade or less", "9 - 12th grade, no diploma", 
           "High school graduate or GED completed", 
           "Some college credit, but no degree", "Associate degree", 
           "Bachelor’s degree", "Master’s degree", 
           "Doctorate or professional degree"]
    return edu[int(s)-1]

df = df.drop(columns=['education_1989_revision', 'education_reporting_flag'])
df = df[df['education_2003_revision'].notnull()]
df['education_2003_revision'] = df['education_2003_revision'].apply(convert_education_2003)

#### Month of Death

In [ ]:
def convert_month(s):
    '''
    Convert the month of death from the encoding to the actual value.
    e.g. 01 => January.
    '''
    months = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]
    return months[s-1]

df['month_of_death'] = df['month_of_death'].apply(convert_month)

#### Place of Death

In [ ]:
def convert_place(s):
    '''
    Convert place of death from the encoding to the actual value.
    e.g. 1 => Hospital, clinic or Medical Center
    '''
    if (s == None or s == 9):
        return None
    place = ["Hospital, Clinic or Medical Center", 
             "Hospital, Clinic or Medical Center",
             "Hospital, Clinic or Medical Center", "Decedent’s home", 
             "Hospice facility", "Nursing home/long term care", "Other"]
    return place[s-1]

df['place_of_death_and_decedents_status'] = df['place_of_death_and_decedents_status'].apply(convert_place)

#### Marital Status

In [ ]:
def convert_marital(s):
    '''
    Convert marital status of decedent from encoding into the actual value.
    e.g. S => Never married, single
    '''
    if (s == None or s == "U"):
        return None
    elif (s == "S"):
        return "Never married, single"
    elif (s == "M"):
        return "Married"
    elif (s == "W"):
        return "Widowed"
    elif (s == "D"):
        return "Divorced"
    
df['marital_status'] = df['marital_status'].apply(convert_marital)

#### Day of Week of Death

In [ ]:
def convert_dow(s):
    '''
    Convert day of week of the death from the encoding to the actual value.
    e.g. 1 => Sunday.
    '''
    if (s == None or s == 9):
        return None
    days = ["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", 
            "Saturday"]
    return days[s-1]

df['day_of_week_of_death'] = df['day_of_week_of_death'].apply(convert_dow)

#### Injury at Work

In [ ]:
def convert_injury(s):
    '''
    Convert whether or not decedent suffered injury from work. Data includes 
    N, Y, None. N - no, Y - yes, None - unknown.
    '''
    if (s == None or s == "U"):
        return None
    return s

df['injury_at_work'] = df['injury_at_work'].apply(convert_injury)

#### Manner of Death

In [ ]:
import math

def convert_manner_of_death(s):
    '''
    Convert the manner of death from the encoding to the actual value.
    e.g. 1 => Accident.
    '''
    manner_of_death={
        "1":"Accident",
        "2":"Suicide",
        "3":"Homicide",
        "4":"Pending investigation",
        "5":"Could not determine",
        "6":"Self-Inflicted",
        "7":"Natural"
    }
    
    if math.isnan(s):
        return "Not Specified"
    else:
        return manner_of_death[str(int(s))]

df['manner_of_death'] = df['manner_of_death'].apply(convert_manner_of_death)

#### Activity Code

In [ ]:
def convert_activity_code(s):
    '''
    Convert the type of activity decedent was engaged in at time of death
    from encoding to the actual value. 
    e.g. 0 => While engaged in sports activity.
    '''
    activity_code={
        "0":"While engaged in sports activity",
        "1":"While engaged in leisure activity",
        "2":"While working for income",
        "3":"While engaged in other types of work",
        "4":"While resting, sleeping, eating (vital activities)",
        "8":"While engaged in other specified activities",
        "9":"During unspecified activity"
    }
    if math.isnan(s):
        return "Not Applicable"
    else:
        return activity_code[str(int(s))]
    return
df['activity_code'] = df['activity_code'].apply(convert_activity_code)

#### Age

In [ ]:
def convert_age_type(s):
    '''
    Convert the type of age in the 'detail_age' column from the encoding the 
    the actual value. 'detail_age' is of type int, so if a row has detail_age 
    of x, then detail_age_type represents what value x is in, e.g. 1 => Years.
    '''
    detail_age_type={
        "1":"Years",
        "2":"Months",
        "4":"Days",
        "5":"Hours",
        "6":"Minutes",
        "9":None
    }
    return detail_age_type[str(s)]
df['detail_age_type'] = df['detail_age_type'].apply(convert_age_type)

We decided to work with the age_recode_12 column since it summarizes the ages into categories that are specific enough to provide a good amount of detail, but not too specific that there are too many unnecessary categories. It's worth noting that the data already has a column named detailed_age and that provides the exact age of the person in question, but we thought it would be more worthwhile to keep this column as well in order to have a more general way of grouping age that we could perform analysis on later. We would likely need to create bins for age later in our visualizations and with this column, that work is already done for us. 

In [ ]:
def convert_age_recode(s):
    '''
    Convert the age group of decedent from the encoding to the actual value.
    e.g. 02 => 1-4 Years.
    '''
    if (s == None):
        return None
    age_recode_12 = {
        10:"75 - 84 years",
        11:"85 years and over",
        12:"Age not stated",
        1:"Under 1 year (includes not stated infant ages)",
        2:"1 - 4 years",
        3:"5 - 14 years",
        4:"15 - 24 years",
        5:"25 - 34 years",
        6:"35 - 44 years",
        7:"45 - 54 years",
        8:"55 - 64 years",
        9:"65 - 74 years"
    }
    return age_recode_12[int(s)]

df['age_recode_12'] = df['age_recode_12'].apply(convert_age_recode)

We then drop the other recodes since we no longer need them. We can also drop the age_substitution_flag column since we don't care if they calculated the age or if it was recorded on the certificate.

In [ ]:
df = df.drop(columns=['age_recode_27', 'age_recode_52', 'infant_age_recode_22', 'age_substitution_flag'])

#### Cause Recode

We chose to use the column 39_cause_recode because we thought the data was an appropriate level of specificity, as opposed to the columns with more or too specific data. We thought that the columns with 358 causes and 113 causes respectively would both be hard to clean appropriately and too specific for the questions we want to ask.

In [ ]:
def convert_cause_recode(s):
    '''
    Convert the cause of death from encoding into the actual value. 
    e.g. 041 => Assault (homicide).
    '''
    cause_recode={
        "001":"Tuberculosis",
        "002":"Syphilis",
        "003":"HIV",
        "004":"Cancer",
        "005":"Stomach cancer",
        "006":"Colorectal cancer",
        "007":"Pancreatic exocrine cancer",
        "008":"Lung and tracheal cancer",
        "009":"Breast cancer",
        "010":"Cervical, ovarian, uterine cancer",
        "011":"Prostate cancer",
        "012":"Urinary tract cancer",
        "013":"Non-Hodgkin's lymphoma",
        "014":"Leukemia",
        "015":"Other cancers",
        "016":"Diabetes mellitus",
        "017":"Alzheimer's disease",
        "018":"Major cardiovascular diseases",
        "019":"Diseases of heart",
        "020":"Hypertensive heart disease",
        "021":"Ischemic heart diseases",
        "022":"Other diseases of heart",
        "023":"Essential (primary) hypertension\nand hypertensive renal disease",
        "024":"Cerebrovascular diseases",
        "025":"Atherosclerosis",
        "026":"Other diseases of circulatory system",
        "027":"Influenza and pneumonia",
        "028":"Chronic lower\nrespiratory diseases",
        "029":"Peptic ulcer",
        "030":"Chronic liver disease\nand cirrhosis",
        "031":"Nephritis, nephrotic syndrome, and nephrosis",
        "032":"Pregnancy, childbirth\nand the puerperium",
        "033":"Certain conditions originating\nin the perinatal period",
        "034":"Congenital malformations, deformations\nand chromosomal abnormalities",
        "035":"Sudden infant\ndeath syndrome",
        "036":"Symptoms, signs and abnormal clinical and laboratory findings, not elsewhere classified (excluding Sudden infant death syndrome)",
        "037":"All other diseases (Residual)",
        "038":"Motor vehicle accidents",
        "039":"All other and unspecified accidents and adverse effects",
        "040":"Intentional self-harm (suicide)",
        "041":"Assault (homicide)",
        "042":"All other external causes"
    }
    key = s
    if key < 10:
        key = "00" + str(key)
    elif key < 100:
        key = "0" + str(key)
    else:
        key = str(key)
    return cause_recode[key]

df['39_cause_recode'] = df['39_cause_recode'].apply(convert_cause_recode)

We then drop the other recodes since we no longer need them.

In [ ]:
df = df.drop(columns=['358_cause_recode', '130_infant_cause_recode', '113_cause_recode'])

Since "All other diseases (Residual)", "All other external causes", "All other and unspecified accidents and adverse effects", and "Symptoms, signs and abnormal clinical and laboratory findings, not elsewhere classified (excluding Sudden infant death syndrome)" are not very descriptive causes of death, we consider dropping rows with these values. 

In [ ]:
causes = ["All other diseases (Residual)", "All other external causes",
          "All other and unspecified accidents and adverse effects",
         "Symptoms, signs and abnormal clinical and laboratory findings, not elsewhere classified (excluding Sudden infant death syndrome)"]
print(df[df['39_cause_recode'].isin(causes)].shape)

(4433813, 24)


Although dropping these rows means the loss of many data observations, we still have so much data left to work with for our visualizations. These rows also contain causes of death that are extremely vague and give no insight into the topic we are trying to examine.

In [ ]:
df = df.loc[df['39_cause_recode'].isin(causes) == False]

#### Autopsy and Manner of Disposal

Since whether or not there was an autopsy and the method of disposition   are events that happen after someone has died, we will drop these columns as they do not give us insight into predicting causes of death or understanding the demographics that suffer from particular causes of death.

In [ ]:
df = df.drop(columns=['autopsy', 'method_of_disposition'])

#### Race and Hispanic Origin

We can see from the data dictionary that some of the race values were imputed, so we want to drop the observations with imputed race since we can't be sure that they are the correct race identities.

In [ ]:
print(df[df['race_imputation_flag'] == 1].shape)
print(df[df['race_imputation_flag'] == 2].shape)
print(df[df['race_imputation_flag'].isnull()].shape)

(39621, 22)
(93122, 22)
(14959178, 22)


We can see that 14959178 rows do not have imputed race values. Thus, dropping these observations with imputed values still results in a significant amount of data to work with.

Thus, we drop these rows as well as the race_imputation_flag column since we no longer need it. We can also drop the columns bridged_race_flag since we don't care if the race was bridged or not for our data analysis.

In [ ]:
df = df[df['race_imputation_flag'].isnull()]
df = df.drop(columns=['race_imputation_flag', 'bridged_race_flag'])

Next, there are multiple columns recording race, so we need to examine them and select which ones are helpful to use for our data analysis.

In [ ]:
print(df[df['race'].notnull()].shape)
print(df[df['race_recode_3'].notnull()].shape)
print(df[df['race_recode_5'].notnull()].shape)

(14959178, 20)
(14959178, 20)
(14959178, 20)


We can see that all three recodes have the same amount of non-null values so dropping any column has the same disadvantage in terms of losing data. We can see from the data dictionary that the race column is the most descriptive and specific. It allows us to examine what patterns may exist within the Asian or Pacific Islander race category specifically. The information contained in both recodes are also just summarizations of the race column, but race_recode_3 categorizes by White, Black, and non-White and non-Black, which is not a very helpful summary. race_recode_5 summarizes the race column into broader categories like White, Black, Asian or Pacific Islander, etc. and could be useful if we want to see mortality among general race groups and not the more specific ones measured by the race column. It saves us the work of having to group the races into broader categories for visualization purposes.

Thus, we retain the race and race_recode_5 columns and drop the race_recode_3 column.

In [ ]:
df = df.drop(columns=['race_recode_3'])

We also need to convert the mapped values of race and race_recode_5 to strings describing the categories based off the data dictionary to make our data analysis more readable.

In [ ]:
def convert_race(s):
    '''
    Takes a number or None representing a race encoding and returns
    a string or None describing the encoded race category.
    '''
    if (s == None):
        return None
    race = {
            18: "Asian Indian", 
            28: "Korean", 
            38: "Samoan", 
            48: "Vietnamese",
            58: "Guamanian", 
            68: "Other Asian or Pacific Islander in areas reporting codes 18-58", 
            78: "Combined other Asian or Pacific Islander, includes codes 18-68",
            1: "White",
            2: "Black",
            3: "American Indian (includes Aleuts and Eskimos)",
            4: "Chinese",
            5: "Japanese",
            6: "Hawaiian (includes Part-Hawaiian)",
            7: "Filipino",
            0: "Other races",
            8: "Other Asian or Pacific Islander"
    }
    return race[s]

def convert_race_recode_5(s):
    '''
    Takes a number or None representing a race encoding and returns
    a string or None describing the encoded race category based on race recode 
    5 rules.
    '''
    if (s == None):
        return None
    race = ["Other (Puerto Rico only)", "White", "Black", "American Indian", 
            "Asian or Pacific Islander"]
    return race[s]

df['race'] = df['race'].apply(convert_race)
df['race_recode_5'] = df['race_recode_5'].apply(convert_race_recode_5)

Since Hispanic is not stored as a value in the race column but rather separately by hispanic origin, we need to clean the data in the hispanic_origin column separately.

In [ ]:
print(df[df['hispanic_origin'].notnull()].shape)
print(df[df['hispanic_originrace_recode'].notnull()].shape)

(14959178, 19)
(14959178, 19)


We can see that both columns have the same amount of non-null values so dropping any column has the same disadvantage in terms of losing data. We can see from the data dictionary that the hispanic_originrace_recode column is just a summarization of the hispanic_origin column, categorizing by broader geographical categories and losing specificity. Based on the data dictionary, it ultimately doesn't tell us the data in a new way that we can't already tell from the hispanic_origin column or the race_column.

Thus, we retain the hispanic_origin column and drop the hispanic_originrace_recode.

In [ ]:
df = df.drop(columns=['hispanic_originrace_recode'])

Finally, we convert the mapped values of hispanic_origin to strings describing the categories based off the data dictionary to make our data analysis more readable.

In [ ]:
def convert_hispanic(s):
    '''
    Takes a number or None representing a hispanic origin encoding and returns
    a string or None describing the encoded hispanic origin category.
    '''
    if (s == None):
        return None
    elif (s == 220):
        return "Central and South American"
    elif (100 <= s and s <= 199): 
        return "Non – Hispanic"
    elif (200 <= s and s <= 209):
        return "Spaniard"
    elif (210 <= s and s <= 219):
        return "Mexican"
    elif (260 <= s and s <= 269):
        return "Puerto Rican"
    elif (270 <= s and s <= 274):
        return "Cuban"
    elif (275 <= s and s <= 279):
        return "Dominican"
    elif (221 <= s and s <= 230):
        return "Central American"
    elif (231 <= s and s <= 249):
        return "South American"
    elif (250 <= s and s <= 259):
        return "Latin American"
    elif (280 <= s and s <= 299):
        return "Other Hispanic"
    elif (996 <= s and s <= 999):
        return "Unknown"

df['hispanic_origin'] = df['hispanic_origin'].apply(convert_hispanic)

Note that we keep race and hispanic origin as separate columns because hispanic is considered to be an attribute describing ethnicity, not race.

We can now see that all of our columns are the correct data type.

In [ ]:
df.dtypes

resident_status                                object
education_2003_revision                        object
month_of_death                                 object
sex                                            object
detail_age_type                                object
detail_age                                      int64
age_recode_12                                  object
place_of_death_and_decedents_status            object
marital_status                                 object
day_of_week_of_death                           object
current_data_year                      datetime64[ns]
injury_at_work                                 object
manner_of_death                                object
activity_code                                  object
39_cause_recode                                object
race                                           object
race_recode_5                                  object
hispanic_origin                                object
dtype: object

We still have null values for some columns but this is fine as we will work around the null values depending on the specific question we are analyzing.

In [ ]:
df.isnull().sum()

resident_status                               0
education_2003_revision                  477475
month_of_death                                0
sex                                           0
detail_age_type                             589
detail_age                                    0
age_recode_12                                 0
place_of_death_and_decedents_status       98883
marital_status                           183014
day_of_week_of_death                        779
current_data_year                             0
injury_at_work                         14075609
manner_of_death                               0
activity_code                                 0
39_cause_recode                               0
race                                          0
race_recode_5                                 0
hispanic_origin                               0
dtype: int64

In [ ]:
df.head()

,resident_status,education_2003_revision,month_of_death,sex,detail_age_type,detail_age,age_recode_12,place_of_death_and_decedents_status,marital_status,day_of_week_of_death,current_data_year,injury_at_work,manner_of_death,activity_code,39_cause_recode,race,race_recode_5,hispanic_origin
124159,RESIDENTS,8th grade or less,June,M,Years,88,85 years and over,Decedent’s home,Married,Saturday,2005-01-01,None,Not Specified,Not Applicable,Ischemic heart diseases,White,White,Mexican
124160,RESIDENTS,"9 - 12th grade, no diploma",January,F,Years,52,45 - 54 years,"Hospital, Clinic or Medical Center",Married,Saturday,2005-01-01,None,Not Specified,Not Applicable,Chronic lower\nrespiratory diseases,White,White,Non – Hispanic
124161,RESIDENTS,High school graduate or GED completed,January,F,Years,70,65 - 74 years,Decedent’s home,Widowed,Sunday,2005-01-01,None,Not Specified,Not Applicable,Lung and tracheal cancer,White,White,Non – Hispanic
124162,RESIDENTS,High school graduate or GED completed,January,M,Years,57,55 - 64 years,Decedent’s home,Married,Monday,2005-01-01,N,Suicide,During unspecified activity,Intentional self-harm (suicide),White,White,Non – Hispanic
124163,RESIDENTS,High school graduate or GED completed,January,M,Years,79,75 - 84 years,Decedent’s home,Married,Sunday,2005-01-01,None,Not Specified,Not Applicable,Cerebrovascular diseases,White,White,Non – Hispanic


In [ ]:
df.to_csv('clean.csv', index=False)